## Extract and save experimental atomic energy levels

In [1]:
import pandas as pd
import numpy as np
import os, re

In [2]:
# Specify the atom and the charge state
atom = 'Co'
charge = 0

In [3]:
astr = atom + '+' + 'i' * (charge + 1)
url = f'https://physics.nist.gov/cgi-bin/ASD/energy1.pl?de=0&spectrum={astr}&submit=Retrieve+Data&units=0&format=0&output=0&page_size=15&multiplet_ordered=0&conf_out=on&term_out=on&level_out=on&unc_out=1&j_out=on&lande_out=on&perc_out=on&biblio=on&temp='

In [4]:
# Read all the tables
dfs = pd.read_html(url, header=0)

In [5]:
# Select the table that contains spectroscopic data
for df in dfs:
    if 'Configuration' in df.columns:
        dfexpt = df

In [6]:
# Keep selected columns
ecol = 'Level (cm-1)'
cols = ['Configuration', 'Term', 'J', ecol]
dfexpt = dfexpt[cols]

In [7]:
# Remove spaces from numbers and convert to float
for irow, estr in enumerate(dfexpt[ecol]):
    try:
        estr = str(estr)  # in case it has already been converted to a number
        e = float(re.sub('\s', '', estr.strip()))
    except ValueError:
        # energy might have "?" or other qualifier; replace with Nan
        print(f'** removing non-numeric energy value "{estr}"')
        e = np.nan
    dfexpt.at[irow, ecol] = e

** removing non-numeric energy value "43 952.06?"
** removing non-numeric energy value "44 381.32?"
** removing non-numeric energy value "48 615.56?"
** removing non-numeric energy value "48 851.58?"
** removing non-numeric energy value "49 197.74?"
** removing non-numeric energy value "52 264.01?"
** removing non-numeric energy value "52 915.92?"
** removing non-numeric energy value "52 264.49?"
** removing non-numeric energy value "52 702.76?"
** removing non-numeric energy value "51 989.31?"
** removing non-numeric energy value "53 694.57?"
** removing non-numeric energy value "54 258.75?"
** removing non-numeric energy value "55 165.63?"
** removing non-numeric energy value "55 577.28?"
** removing non-numeric energy value "54 282.73?"
** removing non-numeric energy value "55 407.10?"
** removing non-numeric energy value "53 728.36?"
** removing non-numeric energy value "54 989.62?"
** removing non-numeric energy value "55 314.04?"
** removing non-numeric energy value "55 120.30?"


In [8]:
# Discard rows that lack energies
dfexpt = dfexpt[dfexpt[ecol].notna()]
# Re-index to make continuous
dfexpt = dfexpt.reset_index(drop=True)
display(dfexpt)

,Configuration,Term,J,Level (cm-1)
0,3p63d74s2,a 4F,9/2,0.0
1,NaN,NaN,7/2,816.0
2,NaN,NaN,5/2,1406.84
3,NaN,NaN,3/2,1809.33
4,3p63d8(3F)4s,b 4F,9/2,3482.82
...,...,...,...,...
302,NaN,36°,7/2,58187.39
303,NaN,NaN,9/2,58187.39
304,NaN,37°,5/2,59388.89
305,NaN,NaN,7/2,59388.89


In [9]:
# Where Configuration or Term is NaN, replace it with value from preceding row
for i, row in dfexpt.iterrows():
    for col in ['Configuration', 'Term']:
        if str(row[col]).lower() == 'nan':
            dfexpt.loc[i, col] = dfexpt.loc[i-1, col]

In [10]:
display(dfexpt)

,Configuration,Term,J,Level (cm-1)
0,3p63d74s2,a 4F,9/2,0.0
1,3p63d74s2,a 4F,7/2,816.0
2,3p63d74s2,a 4F,5/2,1406.84
3,3p63d74s2,a 4F,3/2,1809.33
4,3p63d8(3F)4s,b 4F,9/2,3482.82
...,...,...,...,...
302,3p63d7(a 2D)4s4p(3P°),36°,7/2,58187.39
303,3p63d7(a 2D)4s4p(3P°),36°,9/2,58187.39
304,3p63d7(a 2D)4s4p(3P°),37°,5/2,59388.89
305,3p63d7(a 2D)4s4p(3P°),37°,7/2,59388.89


In [11]:
# Save to Excel file
fxl = atom + '_' + 'I' * (charge + 1) + '_exptl_levels.xlsx'
if os.path.isfile(fxl):
    print(f'File {fxl} already exists!')
else:
    dfexpt.to_excel(fxl, index=False)
    print(f'Data saved to file {fxl}')

Data saved to file Co_I_exptl_levels.xlsx
